In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
nltk.download('punkt')
from tqdm.auto import tqdm
from wordcloud import WordCloud
import html
from langdetect import detect, detect_langs, LangDetectException
from google_trans_new import google_translator
import random
from collections import Counter
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer

# Roadmap to Text Mining

Text mining est le processus d'extraction d'informations à partir de grandes quantités de données non structurées. Cela implique plusieurs étapes, qui peuvent être décomposées en ces étapes suivantes :

1. Collecte des données : La première étape du text mining est la collecte de grande quantité de données non structurées. Cette donnée peut provenir de diverses sources telles que les plateformes sociaux, les sites Web, les blogs et les articles de presse.

2. Prétraitement : Une fois les données recueillies, elles doivent être prétraitées. Cela consiste à nettoyer les données en supprimant les caractères indésirables ou les mots, en corrigeant les erreurs orthographiques et en convertissant le texte en minuscule.

3. Tokenisation : Une fois les données prétraitées, elles sont tokenisées. Cela signifie que le texte est divisé en mots individuels ou des tokens. Cette étape est importante car elle permet d'analyser la fréquence de chaque mot dans le texte.

4. Suppression des mots-clés : Une fois le texte tokenisé, nous pouvons supprimer les mots-clés. Les mots-clés sont des mots courants tels que "le", "et" et "est" qui n'apportent pas de sens particulier et peuvent être supprimés sans affecter l'analyse globale.

5. Reduction morphologique ou normalisation du lemme : La prochaine étape est la reduction morphologique ou la normalisation du lemme. Cela consiste à réduire chaque mot à sa forme de base, également connu sous le nom de racine de mot. Par exemple, "courir" serait réduit à "court". Cela est fait pour regrouper des mots similaires ensemble, comme différents temps verbaux du même mot.

6. Extraction des fonctionnalités : Une fois le texte prétraité et nettoyé, nous pouvons extraire les fonctionnalités à partir de lui. Ces fonctionnalités peuvent inclure des choses telles que la fréquence des mots, l'analyse du sentiment ou le modélisation des sujets.

7. Analyse : Enfin, nous pouvons analyser les données à l'aide de diverses techniques telles que l'agrégation ou la classification pour obtenir des informations sur le texte.

In [ ]:
# CONSTANTES
FIGSIZE = (12, 4)
PATH_DATA = "../data/"
PATH_RAW = PATH_DATA + "raw/"
PATH_PROCESSED = PATH_DATA + "processed/"
PATH_EXTERNAL = PATH_DATA + "external/"

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 200)

In [ ]:
df = pd.read_csv(PATH_RAW + "x_train.csv", index_col=0)
target = pd.read_csv(PATH_RAW + "y_train.csv", index_col=0)
display(df.head())
display(target.describe())
df_origin = pd.read_csv(PATH_RAW + "x_train.csv", index_col=0)

In [ ]:
stop_words_french = pd.read_json(PATH_EXTERNAL + "stop_words_french.json")
print(stop_words_french.shape)
stop_words = []
stop_words.extend(stop_words_french[0].tolist())
stop_words.extend(["cm", "mm"])
print(len(stop_words))
stop_words[:10]

# PRE-PROCESSING LEXIQUE FRANÇAIS

In [ ]:
def strip_accents(texts):
    sentences = []
    for sentence in tqdm(texts):
        if sentence is np.nan:
            sentences.append(np.nan)
        else:
            s = "".join(
                c
                for c in unicodedata.normalize("NFD", sentence)
                if unicodedata.category(c) != "Mn"
            )
            sentences.append(s)
    return sentences

In [ ]:
lexique = pd.read_table(PATH_EXTERNAL + "Lexique383/Lexique383.tsv", delimiter="\t")
lexique_filtred_unique = lexique.drop_duplicates(subset="ortho")[["ortho", "lemme"]]
lexique_filtred_unique.dropna(subset=["ortho", "lemme"], axis="index", inplace=True)
lexique_filtred_unique["ortho"] = strip_accents(lexique_filtred_unique["ortho"])
lexique_filtred_unique["lemme"] = strip_accents(lexique_filtred_unique["lemme"])
lexique_filtred_unique = lexique_filtred_unique.drop_duplicates(subset="ortho")[
    ["ortho", "lemme"]
]

lexique_filtred_unique

# Premier nettoyage
### pour pouvoir traduire les textes d'une autre langue en français

In [ ]:
def first_cleaning(sentences):
    # Pre-compile regex patterns
    HTML_TAGS_RE = re.compile("<[^>]*")
    URL_RE = re.compile(r"https?://[-_.?&~;+=/#0-9A-Za-z]{1,2076}")
    MAIL_RE = re.compile(
        r"[-_.0-9A-Za-z]{1,64}@[-_0-9A-Za-z]{1,255}[-_.0-9A-Za-z]{1,255}"
    )
    SPE_CHAR_RE = re.compile("[^a-zA-ZÀ-ÿ]")
    SPACES_RE = re.compile("\s+")
    SPACES_RE_2 = re.compile(" +")

    sentences_cleaned = []

    for sentence in tqdm(sentences):
        # Decode HTML entities
        sentence = html.unescape(sentence)

        # Replace HTML tags with spaces
        sentence = HTML_TAGS_RE.sub("", sentence)

        # Remove URL and e-mail adress
        sentence = URL_RE.sub("", sentence)
        sentence = MAIL_RE.sub("", sentence)

        # Replace special characters with spaces while keeping accents
        sentence = SPE_CHAR_RE.sub(" ", sentence)

        # Remove unnecessary spaces from the sentence using regular expressions
        cleaned_sentence = SPACES_RE.sub(" ", sentence)
        cleaned_sentence = SPACES_RE_2.sub(" ", cleaned_sentence)
        sentences_cleaned.append(cleaned_sentence)

    return sentences_cleaned

In [ ]:
df["designation"] = first_cleaning(df["designation"])
df["description"].loc[~df["description"].isna()] = first_cleaning(
    df["description"].loc[~df["description"].isna()]
)

# join texts
df["text"] = np.where(
    df["description"].isna(),
    df["designation"].astype(str),
    df["designation"].astype(str) + " " + df["description"].astype(str),
)
df.head()

# Detection des langues